[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/search/multilingual/cohere-multilingual/cohere-multilingual-search.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/search/multilingual/cohere-multilingual/cohere-multilingual-search.ipynb)

In [1]:
!pip install -qU cohere qdrant-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.9/126.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 80.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.3/75.3 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.0 MB/s eta 0:00:00


In [2]:
import os
import cohere
from qdrant_client import QdrantClient
from qdrant_client import models
from qdrant_client.http import models as rest

In [3]:
# load environment variables

QDRANT_API_KEY = "qbcUYNr2s4Q6Mci7i4pMN9KXnd9IiTvDdEWyRba3DaaRn8olA2hDUQ"
COHERE_API_KEY = "DS7FcYkyVyVgxyKxJeoRwLa6EWU76QR5g4YSn6Fv" 

In [8]:
co = cohere.Client(COHERE_API_KEY)

In [4]:
qdrant_client = QdrantClient(
    url="https://abe27875-57da-4af8-85f9-2a50b4aaaf19.us-east-1-0.aws.cloud.qdrant.io:6333", 
    api_key=QDRANT_API_KEY,
    prefer_grpc=True,
)

In [5]:
MLLM_MODEL = "multilingual-22-12"

In [6]:
def query_knowledge_base(question, lang = None):
  query_embeddings = co.embed(
    texts=[question],
    model=MLLM_MODEL,
  )
  query_filter = None


  if lang:
    query_filter = models.Filter(
        must=[
            models.FieldCondition(
                key="lang",
                match=models.MatchValue(
                    value="yo",
                ),
            )
        ]
    )

  result = qdrant_client.search(
    collection_name="wiki-embed",
    query_filter = query_filter, 
    search_params=models.SearchParams(
        hnsw_ef=128,
        exact=False
    ),
    query_vector=query_embeddings.embeddings[0],
    limit=10,
  )

  return result




In [9]:
result = query_knowledge_base("The history of America")

In [ ]:
# https://yo.wikipedia.org/wiki/%C3%8Ct%C3%A0n_il%E1%BA%B9%CC%80_N%C3%A0%C3%ACj%C3%ADr%C3%AD%C3%A0

In [ ]:
# Who is Samuel Ajaji Crowder
# Tell me about Chinua Achebe and his works
# Who was president between 2015 and ----
# I want to know the past presidents of America

In [10]:
for _ in range(10):
  print(result[_].payload["url"])

https://en.wikipedia.org/wiki?curid=3434750
https://en.wikipedia.org/wiki?curid=3434750
https://en.wikipedia.org/wiki?curid=3434750
https://en.wikipedia.org/wiki?curid=3434750
https://en.wikipedia.org/wiki?curid=31717
https://en.wikipedia.org/wiki?curid=3434750
https://en.wikipedia.org/wiki?curid=3434750
https://en.wikipedia.org/wiki?curid=3434750
https://en.wikipedia.org/wiki?curid=3434750
https://en.wikipedia.org/wiki?curid=3434750


In [ ]:
 text = """
Photosynthesis is a process used by plants and other organisms to convert light energy into chemical energy that, through cellular respiration, can later be released to fuel the organism's activities. Some of this chemical energy is stored in carbohydrate molecules, such as sugars and starches, which are synthesized from carbon dioxide and water – hence the name photosynthesis, from the Greek phōs (φῶς), "light", and synthesis (σύνθεσις), "putting together".[1][2][3] Most plants, algae, and cyanobacteria perform photosynthesis; such organisms are called photoautotrophs. Photosynthesis is largely responsible for producing and maintaining the oxygen content of the Earth's atmosphere, and supplies most of the energy necessary for life on Earth.[4]

Although photosynthesis is performed differently by different species, the process always begins when energy from light is absorbed by proteins called reaction centers that contain green chlorophyll (and other colored) pigments/chromophores. In plants, these proteins are held inside organelles called chloroplasts, which are most abundant in leaf cells, while in bacteria they are embedded in the plasma membrane. In these light-dependent reactions, some energy is used to strip electrons from suitable substances, such as water, producing oxygen gas. The hydrogen freed by the splitting of water is used in the creation of two further compounds that serve as short-term stores of energy, enabling its transfer to drive other reactions: these compounds are reduced nicotinamide adenine dinucleotide phosphate (NADPH) and adenosine triphosphate (ATP), the "energy currency" of cells.

In plants, algae and cyanobacteria, sugars are synthesized by a subsequent sequence of light-independent reactions called the Calvin cycle. In the Calvin cycle, atmospheric carbon dioxide is incorporated into already existing organic carbon compounds, such as ribulose bisphosphate (RuBP).[5] Using the ATP and NADPH produced by the light-dependent reactions, the resulting compounds are then reduced and removed to form further carbohydrates, such as glucose. In other bacteria, different mechanisms such as the reverse Krebs cycle are used to achieve the same end.
"""

In [ ]:
response = co.summarize(
    text= text,
    model='summarize-xlarge',
    length='medium',
    format='bullets',
    extractiveness='high',
    temperature=0.3,
    additional_command=None,
)

In [ ]:
response.summary

'- Photosynthesis is a process used by plants and other organisms to convert light energy into chemical energy that, through cellular respiration, can later be released to fuel the organism\'s activities.\n- Some of this chemical energy is stored in carbohydrate molecules, such as sugars and starches, which are synthesized from carbon dioxide and water – hence the name photosynthesis, from the Greek phōs (φῶς), "light", and synthesis (σύνθεσις), "putting together".\n- Most plants, algae, and cyanobacteria perform photosynthesis; such organisms are called photoautotrophs.\n- Photosynthesis is largely responsible for producing and maintaining the oxygen content of the Earth\'s atmosphere, and supplies most of the energy necessary for life on Earth.'

---